In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_teddynote import logging

logging.langsmith("edu-CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
edu-CH01-Basic


### 데이터를 효과적으로 전달하는 방법 

- `RunnablePassthrough`

### RunnablePassthrough

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# prompt와 llm을 생성
prompt = PromptTemplate.from_template("{num} 의 10배는?")
llm = ChatOpenAI(temperature=0)

# chain 생성
chain = prompt | llm

In [ ]:
# chain 실행
chain.invoke({"num": 5})

AIMessage(content='50', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 16, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BFCDi2rXmyjQR77Rl5fF4rwAfG7kw', 'finish_reason': 'stop', 'logprobs': None}, id='run-d86d96b0-461d-44fe-b3ff-d3fbb25f60a4-0', usage_metadata={'input_tokens': 16, 'output_tokens': 2, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
chain.invoke(5)

AIMessage(content='50입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 16, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BFCE6SLy4IU5aW18NJpmhJ5xULwZ7', 'finish_reason': 'stop', 'logprobs': None}, id='run-1032d0b6-2fb1-4d82-a27f-e23e28a8e288-0', usage_metadata={'input_tokens': 16, 'output_tokens': 4, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

`RunnablePassthrough`를 사용한 예제

In [6]:
from langchain_core.runnables import RunnablePassthrough

# runnable
RunnablePassthrough().invoke({"num": 10})

{'num': 10}

`RunnablePassthrough`로 체인 구성 예제

In [ ]:
# 기존 체인 생성 및 값 주입 방식
chain = prompt | ChatOpenAI()
chain.invoke({"num": 5})

# RunnablePassthrough 사용한 체인 생성 및 값 주입 방식
chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()
chain.invoke(5)

In [ ]:
# 사용자 입력값(10)이 RunnablePassthrough()로 전달됨
runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()

# dict 값이 RunnablePassthrough() 변경됨
runnable_chain.invoke(10)

AIMessage(content='10의 10배는 100입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 16, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BFCJZIVbCFf6E71EKSA4cO3EfPNBj', 'finish_reason': 'stop', 'logprobs': None}, id='run-5daa6eb1-6084-4296-b025-3c87677e37ba-0', usage_metadata={'input_tokens': 16, 'output_tokens': 12, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

`RunnablePassthrough.assign()` 을 사용하는 에제

In [8]:
# 입력 키: num, 할당(assign) 키: new_num
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

{'num': 1, 'new_num': 3}

### RunnableParallel

In [ ]:
from langchain_core.runnables import RunnableParallel

# RunnableParallel 인스턴스 생성
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(multi=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

# runnable 인스턴스에 {'num': 1} 딕셔너리를 입력으로 전달하여 invoke 메소드 호출
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'multi': 3}, 'modified': 2}

Chain도 RunnableParallel 적용 가능 

In [ ]:
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | ChatOpenAI()
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | ChatOpenAI()
)

In [13]:
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")

{'capital': AIMessage(content='서울입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 19, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BFCeuqncdhXqLTNgtj9f3uka0AkRH', 'finish_reason': 'stop', 'logprobs': None}, id='run-c4019d52-e5a1-404e-8f27-6923912cbc03-0', usage_metadata={'input_tokens': 19, 'output_tokens': 6, 'total_tokens': 25, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'area': AIMessage(content='대한민국의 면적은 약 100,363km² 입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 20, 'total_tokens': 43, 'completion_tokens_details': {

### RunnableLambda

사용자 정의 함수 맵핑

In [27]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime


def get_today(a):
    print(f"입력받은 변수 a의 값: {a}")
    print(f"입력받은 n의 값: {a['n']}")
    # 오늘 날짜 가져오기
    return datetime.today().strftime("%b-%d")


# 오늘 날짜로 출력
# get_today(None)

In [28]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

# prompt 와 llm 생성
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# chain 생성
chain = (
    # case 1
    # {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    # case 2
    {"today": RunnableLambda(get_today), "n": itemgetter("n")}
    | prompt
    | llm
    | StrOutputParser()
)

In [29]:
# 출력

# case 1
# print(chain.invoke(3))

# case 2
print(chain.invoke({"n": 3}))

입력받은 변수 a의 값: {'n': 3}
입력받은 n의 값: 3
3월 26일이 생일인 유명인 세 명은 다음과 같습니다.

1. 레너드 니모이 (Leonard Nimoy) - 1931년 3월 26일
2. 다이애나 로스 (Diana Ross) - 1944년 3월 26일
3. 스티븐 타일러 (Steven Tyler) - 1948년 3월 26일

이들은 각각 배우, 가수, 음악가로 잘 알려져 있습니다.


`itemgetter`를 사용하여 특정 키를 추출할 수 있음

In [ ]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


# 문장의 길이를 반환하는 함수
def length_function(text):
    return len(text)


# 두 문장의 길이를 곱한 값을 반환하는 함수
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


# _multiple_length_function 함수를 사용하여 두 문장의 길이를 곱한 값을 반환하는 함수
# RunnableLambda 함수는 무조건 매개변수를 1개만 받아야 하므로 딕셔너리 타입으로 2개 이상의 값을 전달하도록 구성한 것임.
def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("{a} + {b} 는 무엇인가요?")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [31]:
chain.invoke({"word1": "hello", "word2": "world"})

AIMessage(content='5 + 25는 30입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 22, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BFDApIxToeBQVqEPvg85KOxWMG5EY', 'finish_reason': 'stop', 'logprobs': None}, id='run-fffb4886-dc9e-4a53-bfa7-4dfb127460e5-0', usage_metadata={'input_tokens': 22, 'output_tokens': 10, 'total_tokens': 32, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})